In [35]:
from pyspark.sql import Window
from pyspark.sql.functions import lag, col, row_number, lit
from sparkstudy.deploy.demo_sessions import DemoSQLSessionFactory
from sparkstudy.libs.tools import create_random_data
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
COLUMNS = ["name","age","salary"]
session_factory_normal = DemoSQLSessionFactory(name="normal")
data = create_random_data(10)
spark_session = session_factory_normal.build_session()
df = spark_session.createDataFrame(data,COLUMNS).cache()
df.show()

+----+---+-------------------+
|name|age|             salary|
+----+---+-------------------+
|   Q|  1| 0.6421234209606654|
|   H|  4| 0.2223963428660244|
|   A|  1| 0.2953065373867898|
|   F|  3|0.18001048276677545|
|   E|  5|0.04887312937966426|
|   O| 10|0.04038771202528457|
|   J|  7|  0.129799869320246|
|   S|  8| 0.6409452812403059|
|   B|  7| 0.5394980172932448|
|   Z|  6|  0.320494652358459|
+----+---+-------------------+



本来想要实验一下怎么shift的。结果发现在spark里面，这类操作必须通过window

In [37]:
w = Window().partitionBy("name").orderBy(col("name"))
df.select("*", lag("age").over(w).alias("new_col")).show()

+----+---+-------------------+-------+
|name|age|             salary|new_col|
+----+---+-------------------+-------+
|   F|  3|0.18001048276677545|   null|
|   Q|  1| 0.6421234209606654|   null|
|   E|  5|0.04887312937966426|   null|
|   B|  7| 0.5394980172932448|   null|
|   O| 10|0.04038771202528457|   null|
|   J|  7|  0.129799869320246|   null|
|   Z|  6|  0.320494652358459|   null|
|   A|  1| 0.2953065373867898|   null|
|   S|  8| 0.6409452812403059|   null|
|   H|  4| 0.2223963428660244|   null|
+----+---+-------------------+-------+



In [38]:
w = Window().partitionBy().orderBy(lit(1))
df.select("*", lag("age",2).over(w).alias("new_col")).show()

+----+---+-------------------+-------+
|name|age|             salary|new_col|
+----+---+-------------------+-------+
|   Q|  1| 0.6421234209606654|   null|
|   H|  4| 0.2223963428660244|   null|
|   A|  1| 0.2953065373867898|      1|
|   F|  3|0.18001048276677545|      4|
|   E|  5|0.04887312937966426|      1|
|   O| 10|0.04038771202528457|      3|
|   J|  7|  0.129799869320246|      5|
|   S|  8| 0.6409452812403059|     10|
|   B|  7| 0.5394980172932448|      7|
|   Z|  6|  0.320494652358459|      8|
+----+---+-------------------+-------+

